In [511]:
# Import necessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import (
    PowerTransformer,
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
)
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, chi2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pickle
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE

from scipy import stats
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.pipeline import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import re
from sklearn.impute import SimpleImputer

from ydata_profiling import ProfileReport
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score

In [512]:
# Load data
excel_file_path = "./train.csv"
df = pd.read_csv(excel_file_path, encoding="latin-1")

In [ ]:
def gen_eda():
    profile = ProfileReport(
        pd.concat([df], axis=1),
        title="Pandas Profiling Report",
        explorative=True,
    )
    profile.to_file("pandas_profiling_report.html")


# gen_eda()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\typeset.py:208: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_valid_dtype = pdt.is_categorical_dtype(series) and not pdt.is_bool_dtype(
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\typeset_relations.py:34: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pdt.is_categorical_dtype(series):
c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\t

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [514]:
df.sample(5)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
565261,565261,Melody Mix,Episode 81,105.96,Music,54.92,Friday,Afternoon,80.76,0.0,Neutral,99.09384
75370,75370,Laugh Line,Episode 62,71.13,Comedy,31.86,Tuesday,Afternoon,55.94,0.0,Positive,62.84278
152790,152790,True Crime Stories,Episode 41,41.32,True Crime,56.34,Friday,Evening,17.75,2.0,Negative,30.30636
25209,25209,Sports Weekly,Episode 32,113.54,Sports,59.37,Tuesday,Morning,69.64,1.0,Neutral,79.79183
116642,116642,Educational Nuggets,Episode 35,18.24,Education,54.90,Monday,Evening,50.96,3.0,Neutral,3.85814


In [515]:
df.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,750000.000000,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,374999.500000,64.504738,59.859901,52.236449,1.348855,45.437406
std,216506.495284,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000
25%,187499.750000,35.730000,39.410000,28.380000,0.000000,23.178350
50%,374999.500000,63.840000,60.050000,53.580000,1.000000,43.379460
75%,562499.250000,94.070000,79.530000,76.600000,2.000000,64.811580
max,749999.000000,325.240000,119.460000,119.910000,103.910000,119.970000


In [516]:
df = df.dropna(subset=[
    "Episode_Length_minutes",
    # "Guest_Popularity_percentage"
])
print(df["Episode_Length_minutes"].isnull().sum(), df["Guest_Popularity_percentage"].isnull().sum())

0 123858


In [517]:
def remove_outliers(df, outlier_dict):
    for distribution, category in outlier_dict.items():
        if distribution == "normal":
            for cat in category:
                upper_limit = df[cat].mean() + 3 * df[cat].std()
                lower_limit = df[cat].mean() - 3 * df[cat].std()
                print(cat, upper_limit, lower_limit)
                # capping
                # df[cat] = np.where(df[cat] > upper_limit,upper_limit,np.where(df[cat] < lower_limit, lower_limit, df[cat]))
                # Trimming
                df = df[(df[cat] < upper_limit) & (df[cat] > lower_limit)]
        elif distribution == "skew":
            for cat in category:
                percentile25 = df[cat].quantile(0.4)
                percentile75 = df[cat].quantile(0.75)
                iqr = percentile75 - percentile25
                upper_limit = percentile75 + 1.5 * iqr
                lower_limit = percentile25 - 1.5 * iqr
                print(cat, upper_limit, lower_limit)
                # capping
                # df[cat] = np.where(
                #     df[cat] > upper_limit,
                #     upper_limit,
                #     np.where(df[cat] < lower_limit, lower_limit, df[cat]),
                # )
                # Trimming
                df = df[(df[cat] < upper_limit) & (df[cat] > lower_limit)]
    return df

In [518]:
outlier_dict = {
    "normal": [],
    "skew": [],
}


def pre_process(df):
    df["Publication_Day"] = (
        df["Publication_Day"]
        .map(
            {
                "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7,
            }
        )
        .fillna(0)
        .astype(int)
    )
    df["Publication_Time"] = (
        df["Publication_Time"]
        .map({"Morning": 1, "Afternoon": 2, "Evening": 3, "Night": 4})
        .fillna(0)
        .astype(int)
    )
    df["Episode_Sentiment"] = (
        df["Episode_Sentiment"]
        .map({"Negative": 0, "Neutral": 1, "Positive": 2})
        .fillna(0)
        .astype(int)
    )
    return df


df = pre_process(df)
df = remove_outliers(df, outlier_dict)

In [519]:
df.to_csv("df.csv", index=False)

In [520]:
# Define features and target
def get_X_Y(df):
    X = df.drop(columns=["id", "Listening_Time_minutes", "Episode_Title"])
    Y = df["Listening_Time_minutes"]
    return X, Y


X, Y = get_X_Y(df)
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=5
)
print(X_train.shape)

(530325, 9)


In [521]:
# Get the list of categorical column names
categories_order = {
    "Publication_Day": sorted(list(df["Publication_Day"].unique())),
    "Publication_Time": sorted(list(df["Publication_Time"].unique())),
    "Episode_Sentiment": sorted(list(df["Episode_Sentiment"].unique())),
}
categorical_feat_ord = list(categories_order.keys())
categorical_feat_nom = [ "Podcast_Name", "Genre"]
categorical = categorical_feat_nom + categorical_feat_ord
numerical_features = [col for col in X_train.columns if col not in categorical]

In [522]:
# Separate transformers for categorical and numerical features

# trf = FunctionTransformer(np.log1p, validate=True)
# trf = PowerTransformer()
# trf = FunctionTransformer(np.sqrt, validate=True)
# trf = FunctionTransformer(np.sin)
trf = StandardScaler()
# trf = MinMaxScaler()
trf = FunctionTransformer(np.log1p, validate=True)
# Add Polynomial Features
poly = PolynomialFeatures(degree=2, include_bias=False)

numerical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("poly", poly), ("log", trf)]
)
categorical_transformer_onehot = Pipeline(
    steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)
categorical_transformer_ordinal = Pipeline(
    steps=[
        ("ord", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ]
)

In [523]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer_onehot, categorical_feat_nom),
        ("cat_1", categorical_transformer_ordinal, categorical_feat_ord),
        ("num", numerical_transformer, numerical_features),
    ]
)

from sklearn.linear_model import Lasso, LinearRegression

model = LinearRegression()
model = Lasso(alpha=0.1)
# model = XGBRegressor(objective='reg:squarederror', scale_pos_weight=1)

pipeline = Pipeline([("preprocessor", preprocessor), ("model", model)])

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Podcast_Name', 'Genre']),
                                                 ('cat_1',
                                                  Pipeline(steps=[('ord',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['Publication_Day',
                                                   'Publication_Time',
                                                   'Episode_Sentiment']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('poly',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('log',
                                                                   FunctionTransformer(func=<ufunc 'log1p'>,
                                                                                       validate=True))]),
                                                  ['Episode_Length_minutes',
                                                   'Host_Popularity_percentage',
                                                   'Guest_Popularity_percentage',
                                                   'Number_of_Ads'])])),
                ('model', Lasso(alpha=0.1))])

In [524]:
# # Calculate the correlation matrix
# correlation_matrix = df.corr()

# # Save the correlation matrix to a CSV file
# correlation_matrix.to_csv('correlation_matrix.csv', index=True)

In [525]:
# Save the fitted pipeline as a .pkl file
filename_pkl = "model.pkl"
pickle.dump(pipeline, open(filename_pkl, "wb"))
print(f"Model saved as {filename_pkl}")

# Evaluate the model
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")


def adjusted_r2_score(y_true, y_pred, X):
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)  # number of samples
    k = X.shape[1]  # number of features
    adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - k - 1))
    return r2, adj_r2


# Example usage:
r2, adj_r2 = adjusted_r2_score(Y_test, y_pred, X_test)
print(f"R² Score: {r2:.4f}")
print(f"Adjusted R² Score: {adj_r2:.4f}")

# Define the columns expected by the model
column_names = X_train.columns


def generate_submission(test_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(test_file)
    df = pd.DataFrame(df)
    # Replace empty strings with NaN
    df.replace("", np.nan, inplace=True)
    df = pre_process(df)
    # Select the relevant columns
    filtered_df = df[column_names]
    predictions = pipeline.predict(filtered_df)
    # Load the original test file to keep the PassengerId column
    original_df = pd.read_csv(test_file)
    original_df["Listening_Time_minutes"] = predictions
    original_df["Listening_Time_minutes"] = np.expm1(
        original_df["Listening_Time_minutes"]
    )
    # Save the results to a new CSV file
    submission_df = original_df[["id", "Listening_Time_minutes"]]
    submission_df.to_csv("submission.csv", index=False)
    print("Submission file saved as 'submission.csv'")


# Generate the submission
test_file = "test.csv"
generate_submission(test_file)

Model saved as model.pkl
Mean Squared Error: 180.20351362896974
Root Mean Squared Error: 13.423990227535542
R² Score: 0.7578
Adjusted R² Score: 0.7578
Submission file saved as 'submission.csv'
